# Ejercicio 3. Modelos de Regresión

In [ ]:
!pip install xgboost statsmodels tensorflow

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from itertools import combinations
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import acorr_ljungbox
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam


In [ ]:
df_viento = pd.read_csv("https://raw.githubusercontent.com/lihkir/Data/refs/heads/main/wind_speed/data_treino_dv_df_2000_2010.csv")

df_viento.columns = [
    "hora", "viento_direccion_horaria", "viento_velocidad_horaria",
    "humedad_max_hora", "humedad_min_hora", "temp_min_hora", "temp_max_hora",
    "humedad_relativa", "presion_armosferica", "precip_atmos_total", "viento_bajada_max",
    "precip_atmos_max", "precip_atmos_min"
]

df_viento

,hora,viento_direccion_horaria,viento_velocidad_horaria,humedad_max_hora,humedad_min_hora,temp_min_hora,temp_max_hora,humedad_relativa,presion_armosferica,precip_atmos_total,viento_bajada_max,precip_atmos_max,precip_atmos_min
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2
2,14:00,0.891007,2.0,56.0,50.0,25.5,24.3,51.0,888.1,0.0,4.9,888.4,888.1
3,15:00,0.848048,2.5,52.0,44.0,27.4,25.0,44.0,887.4,0.0,5.8,888.1,887.4
4,16:00,0.224951,2.4,50.0,43.0,27.1,25.5,46.0,886.5,0.0,5.8,887.4,886.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87688,19:00,-0.615661,5.6,83.0,78.0,21.8,21.1,80.0,879.1,0.0,12.3,879.8,879.1
87689,20:00,-0.469472,4.9,84.0,79.0,21.7,21.0,84.0,879.2,0.0,9.9,879.2,878.9
87690,21:00,-0.484810,4.5,86.0,82.0,21.2,20.6,86.0,879.7,0.0,8.9,879.8,879.2
87691,22:00,-0.484810,3.2,88.0,85.0,20.6,20.2,88.0,880.5,0.0,8.0,880.5,879.6


In [ ]:
df_viento_sin_hora = df_viento.drop(columns=["hora"])

X = df_viento_sin_hora.drop(columns=["viento_velocidad_horaria"]).to_numpy(dtype=float)
y = df_viento_sin_hora["viento_velocidad_horaria"].to_numpy(dtype=float)

ventanas = [7, 14, 21, 28]

In [ ]:
y

array([1.8, 2.7, 2. , ..., 4.5, 3.2, 2.3])

In [ ]:
def pliegues_rolling(dias, muestras_dia=24):
    w = dias * muestras_dia
    t = muestras_dia
    for fin in range(w, len(y) - t + 1, t):
        yield np.arange(fin - w, fin), np.arange(fin, fin + t)


def obtener_predicciones(modelo, dias):
    yv, yp = [], []
    for tr, te in pliegues_rolling(dias):
        m = clone(modelo).fit(X[tr], y[tr])
        p = m.predict(X[te])
        yv.append(y[te])
        yp.append(p)
    return np.concatenate(yv), np.concatenate(yp)


def metricas(yv, yp):
    resid = yv - yp
    mape = np.mean(np.abs((yv - yp) / np.where(yv == 0, 1e-8, yv))) * 100
    mae = mean_absolute_error(yv, yp)
    rmse = np.sqrt(mean_squared_error(yv, yp))
    mse = mean_squared_error(yv, yp)
    r2 = r2_score(yv, yp)

    resid -= resid.mean()
    pval = acorr_ljungbox(resid, lags=10, return_df=True)['lb_pvalue'].iloc[-1]
    return mape, mae, rmse, mse, r2, np.format_float_scientific(pval, precision=3)




### Codigo de KNN

In [ ]:
# SECCIÓN 1 ─ KNN ENTRENAMIENTO
modelo_knn = Pipeline([("esc", StandardScaler()), ("knn", KNeighborsRegressor(n_neighbors=3))])
y_true_knn, y_pred_knn = {}, {}
for v in ventanas:
    t0 = time.time()
    yt, yp = obtener_predicciones(modelo_knn, v)
    y_true_knn[v], y_pred_knn[v] = yt, yp
    print(f"KNN ventana {v} días {time.time()-t0:.2f}s")

KNN ventana 7 días 8.71s
KNN ventana 14 días 9.51s
KNN ventana 21 días 10.58s
KNN ventana 28 días 11.15s


KNN ventana 7 días 8.71s

KNN ventana 14 días 9.51s

KNN ventana 21 días 10.58s

KNN ventana 28 días 11.15s

In [ ]:
# SECCIÓN 2 ─ KNN MÉTRICAS
res_knn = {v: metricas(y_true_knn[v], y_pred_knn[v]) for v in ventanas}
df_knn = pd.DataFrame(res_knn, index=["MAPE","MAE","RMSE","MSE","R2","LB_p"]).T
print(df_knn)
print("Mejor ventana KNN", df_knn.RMSE.idxmin())
df_knn

        MAPE       MAE      RMSE       MSE        R2    LB_p
7  2.487e+08 6.745e-01 8.753e-01 7.661e-01 5.564e-01  0.e+00
14 2.318e+08 6.282e-01 8.164e-01 6.665e-01 6.143e-01  0.e+00
21 2.245e+08 6.055e-01 7.877e-01 6.205e-01 6.409e-01  0.e+00
28 2.166e+08 5.926e-01 7.705e-01 5.937e-01 6.565e-01  0.e+00
Mejor ventana KNN 28


,MAPE,MAE,RMSE,MSE,R2,LB_p
7,2.487e+08,6.745e-01,8.753e-01,7.661e-01,5.564e-01,0.e+00
14,2.318e+08,6.282e-01,8.164e-01,6.665e-01,6.143e-01,0.e+00
21,2.245e+08,6.055e-01,7.877e-01,6.205e-01,6.409e-01,0.e+00
28,2.166e+08,5.926e-01,7.705e-01,5.937e-01,6.565e-01,0.e+00


```

        MAPE       MAE      RMSE       MSE        R2    LB_p
7  2.487e+08 6.745e-01 8.753e-01 7.661e-01 5.564e-01  0.e+00
14 2.318e+08 6.282e-01 8.164e-01 6.665e-01 6.143e-01  0.e+00
21 2.245e+08 6.055e-01 7.877e-01 6.205e-01 6.409e-01  0.e+00
28 2.166e+08 5.926e-01 7.705e-01 5.937e-01 6.565e-01  0.e+00

Mejor ventana KNN 28

MAPE	MAE	RMSE	MSE	R2	LB_p
7	2.487e+08	6.745e-01	8.753e-01	7.661e-01	5.564e-01	0.e+00
14	2.318e+08	6.282e-01	8.164e-01	6.665e-01	6.143e-01	0.e+00
21	2.245e+08	6.055e-01	7.877e-01	6.205e-01	6.409e-01	0.e+00
28	2.166e+08	5.926e-01	7.705e-01	5.937e-01	6.565e-01	0.e+00
```

  <div style="background-color: #f5fdf9; padding: 12px;">
    <h3><strong>💡 Análisis del Modelo KNN</strong></h3>
    <p>Los resultados del modelo KNN muestran una mejora gradual en el rendimiento a medida que aumenta el tamaño de la ventana de predicción (T). A continuación, se detalla el análisis de las métricas:</p>
    <hr>
    <h4><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h4>
    <ul>
      <li><strong>MAPE (Mean Absolute Percentage Error):</strong><br>
        Aunque el MAPE disminuye con ventanas más grandes, los valores siguen siendo muy altos (del orden de 10⁸), lo que indica un gran error porcentual en las predicciones.<br>
        T=7: <code>2.487e+08</code><br>
        T=28: <code>2.166e+08</code> (mejora del 12.9%)
      </li>
      <li><strong>MAE (Mean Absolute Error):</strong><br>
        El error absoluto disminuye con ventanas más grandes:<br>
        T=7: <code>6.745e-01</code><br>
        T=28: <code>5.926e-01</code> (mejora del 12.0%)
      </li>
      <li><strong>RMSE (Root Mean Squared Error):</strong><br>
        Similar al MAE, el RMSE también mejora con ventanas más grandes:<br>
        T=7: <code>8.753e-01</code><br>
        T=28: <code>7.705e-01</code> (mejora del 11.8%)
      </li>
      <li><strong>MSE (Mean Squared Error):</strong><br>
        El error cuadrático medio sigue la misma tendencia que RMSE:<br>
        T=7: <code>7.661e-01</code><br>
        T=28: <code>5.937e-01</code> (mejora del 22.4%)
      </li>
      <li><strong>R2 (R-squared):</strong><br>
        El coeficiente de determinación mejora con ventanas más grandes, aunque sigue siendo relativamente bajo:<br>
        T=7: <code>5.564e-01</code> (R2=0.556)<br>
        T=28: <code>6.565e-01</code> (R2=0.656)
      </li>
    </ul>
    <hr>
    <h4><strong>2. Problemas Identificados</strong></h4>
    <ul>
      <li><strong>Altos Valores de MAPE:</strong><br>
        Los valores de MAPE son extremadamente altos en todas las ventanas, lo que podría indicar:
        <ul>
          <li>Problemas de escala</li>
          <li>Ruido en los datos</li>
         </ul>
      </li>
      <li><strong>Autocorrelación en Residuos:</strong><br>
        El valor <code>LB_p = 0.e+00</code> en todas las ventanas, lo que indica autocorrelación significativa en los residuos. Esto viola los supuestos de regresión y sugiere que el modelo no captura la dinámica temporal de los datos.
      </li>
    </ul>
    <hr>
    <h4><strong>3. Mejores Resultados</strong></h4>
    <p>La ventana de tamaño <strong>T=28</strong> muestra los mejores resultados:</p>
    <ul>
      <li>Menor MAPE: <code>2.166e+08</code></li>
      <li>Menor RMSE: <code>7.705e-01</code></li>
      <li>Mayor R2: <code>6.565e-01</code></li>
    </ul>
    <hr>
    <h4><strong>4. Recomendaciones</strong></h4>
    <ol>
      <li><strong>Mejorar el Modelo KNN:</strong>
        <ul>
          <li>Ajustar el parámetro <code>n_neighbors</code></li>
          <li>Explorar la ponderación por distancia <code>weights='distance'</code></li>
        </ul>
      </li>
      <li><strong>Reducir el MAPE:</strong>
        <ul>
          <li>Revisar las unidades de las variables</li>
          <li>Considerar la transformación de la variable objetivo</li>
        </ul>
      </li>
    </ol>
    <hr>
    <h4><strong>Conclusión</strong></h4>
    <p>Aunque la ventana <strong>T=28</strong> muestra el mejor rendimiento, los errores siguen siendo muy altos, lo que sugiere que el modelo KNN puede no ser el más adecuado.</p>
  </div>
</div>

### Codigo de Regresion Lineal

In [ ]:
# SECCIÓN 3 ─ REGRESIÓN LINEAL ENTRENAMIENTO
modelo_lr = LinearRegression()
y_true_lr, y_pred_lr = {}, {}
for v in ventanas:
    t0 = time.time()
    yt, yp = obtener_predicciones(modelo_lr, v)
    y_true_lr[v], y_pred_lr[v] = yt, yp
    print(f"LR ventana {v} días {time.time()-t0:.2f}s")

LR ventana 7 días 3.69s
LR ventana 14 días 3.75s
LR ventana 21 días 3.94s
LR ventana 28 días 4.11s


LR ventana 7 días 3.69s

LR ventana 14 días 3.75s

LR ventana 21 días 3.94s

LR ventana 28 días 4.11s

In [ ]:
# SECCIÓN 4 ─ REGRESIÓN LINEAL MÉTRICAS
res_lr = {v: metricas(y_true_lr[v], y_pred_lr[v]) for v in ventanas}
df_lr = pd.DataFrame(res_lr, index=["MAPE","MAE","RMSE","MSE","R2","LB_p"]).T
print(df_lr)
print("Mejor ventana LR", df_lr.RMSE.idxmin())

df_lr

        MAPE       MAE      RMSE       MSE        R2      LB_p
7  1.646e+08 4.975e-01 1.282e+00 1.643e+00 4.891e-02 0.000e+00
14 1.545e+08 4.733e-01 1.013e+00 1.027e+00 4.057e-01 0.000e+00
21 1.519e+08 4.612e-01 8.246e-01 6.800e-01 6.065e-01 0.000e+00
28 1.518e+08 4.584e-01 7.998e-01 6.397e-01 6.299e-01 0.000e+00
Mejor ventana LR 28


,MAPE,MAE,RMSE,MSE,R2,LB_p
7,1.646e+08,4.975e-01,1.282e+00,1.643e+00,4.891e-02,0.000e+00
14,1.545e+08,4.733e-01,1.013e+00,1.027e+00,4.057e-01,0.000e+00
21,1.519e+08,4.612e-01,8.246e-01,6.800e-01,6.065e-01,0.000e+00
28,1.518e+08,4.584e-01,7.998e-01,6.397e-01,6.299e-01,0.000e+00


```
        MAPE       MAE      RMSE       MSE        R2      LB_p
7  1.646e+08 4.975e-01 1.282e+00 1.643e+00 4.891e-02 0.000e+00
14 1.545e+08 4.733e-01 1.013e+00 1.027e+00 4.057e-01 0.000e+00
21 1.519e+08 4.612e-01 8.246e-01 6.800e-01 6.065e-01 0.000e+00
28 1.518e+08 4.584e-01 7.998e-01 6.397e-01 6.299e-01 0.000e+00

Mejor ventana LR 28

MAPE	MAE	RMSE	MSE	R2	LB_p
7	1.646e+08	4.975e-01	1.282e+00	1.643e+00	4.891e-02	0.000e+00
14	1.545e+08	4.733e-01	1.013e+00	1.027e+00	4.057e-01	0.000e+00
21	1.519e+08	4.612e-01	8.246e-01	6.800e-01	6.065e-01	0.000e+00
28	1.518e+08	4.584e-01	7.998e-01	6.397e-01	6.299e-01	0.000e+00

```

  <div style="background-color: #f5fdf9; padding: 12px;">
    <h3><strong>💡Análisis del Modelo de Regresión Lineal</strong></h3>
    <p>A continuación, se presenta el análisis de los resultados obtenidos con el modelo de Regresión Lineal para diferentes tamaños de ventana (T = 7, 14, 21, 28).</p>
    <hr>
    <h3><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h3>
    <h4><strong>1.1. Reducción del Error</strong></h4>
    <ul>
      <li><strong>MAPE (Mean Absolute Percentage Error):</strong> El MAPE disminuye ligeramente con el aumento del tamaño de la ventana:
        <ul>
          <li>T=7: <code>1.646e+08</code></li>
          <li>T=28: <code>1.518e+08</code> (reducción del 7.8%)</li>
        </ul>
      </li>
      <li><strong>MAE (Mean Absolute Error):</strong> El error absoluto también disminuye:
        <ul>
          <li>T=7: <code>4.975e-01</code></li>
          <li>T=28: <code>4.584e-01</code> (reducción del 7.9%)</li>
        </ul>
      </li>
      <li><strong>RMSE (Root Mean Squared Error):</strong> El error cuadrático medio muestra una mejora significativa:
        <ul>
          <li>T=7: <code>1.282e+00</code></li>
          <li>T=28: <code>7.998e-01</code> (reducción del 37.7%)</li>
        </ul>
      </li>
      <li><strong>MSE (Mean Squared Error):</strong> El error cuadrático medio sigue la misma tendencia:
        <ul>
          <li>T=7: <code>1.643e+00</code></li>
          <li>T=28: <code>6.397e-01</code> (reducción del 61.0%)</li>
        </ul>
      </li>
    </ul>
    <h4><strong>1.2. Mejora en el Coeficiente de Determinación (R2)</strong></h4>
    <ul>
      <li><strong>R2:</strong> El coeficiente de determinación aumenta con el tamaño de la ventana:
        <ul>
          <li>T=7: <code>4.891e-02</code> (R2=0.049)</li>
          <li>T=28: <code>6.299e-01</code> (R2=0.629)</li>
        </ul>
      </li>
    </ul>
    <hr>
    <h3><strong>2. Comparación con el Modelo KNN</strong></h3>
    <ul>
      <li><strong>MAPE:</strong> El modelo de Regresión Lineal muestra un MAPE significativamente menor:
        <ul>
          <li>T=28: <code>1.518e+08</code> (Regresión Lineal) vs. <code>2.166e+08</code> (KNN)</li>
        </ul>
      </li>
      <li><strong>R2:</strong> Aunque mejora, sigue ligeramente por debajo del modelo KNN:
        <ul>
          <li>T=28: <code>6.299e-01</code> (Regresión Lineal) vs. <code>6.565e-01</code> (KNN)</li>
        </ul>
      </li>
    </ul>
    <hr>
    <h3><strong>3. Ventana Óptima</strong></h3>
    <p>La ventana de tamaño <strong>T=28</strong> muestra los mejores resultados:</p>
    <ul>
      <li><strong>MAPE más bajo:</strong> <code>1.518e+08</code></li>
      <li><strong>RMSE más bajo:</strong> <code>7.998e-01</code></li>
      <li><strong>MSE más bajo:</strong> <code>6.397e-01</code></li>
      <li><strong>R2 más alto:</strong> <code>6.299e-01</code></li>
    </ul>
    <hr>
    <h3><strong>Conclusión</strong></h3>
    <p>El modelo de Regresión Lineal mejora sus métricas con ventanas mayores y supera al modelo KNN en MAPE y RMSE, pero su capacidad de capturar la dinámica temporal sigue siendo limitada.</p>

  </div>
</div>


### Codigo de Ridge

In [ ]:
# SECCIÓN 5 ─ RIDGE ENTRENAMIENTO
modelo_ridge = Ridge()
y_true_ridge, y_pred_ridge = {}, {}
for v in ventanas:
    t0 = time.time()
    yt, yp = obtener_predicciones(modelo_ridge, v)
    y_true_ridge[v], y_pred_ridge[v] = yt, yp
    print(f"Ridge ventana {v} días {time.time()-t0:.2f}s")


Ridge ventana 7 días 4.03s
Ridge ventana 14 días 4.18s
Ridge ventana 21 días 4.18s
Ridge ventana 28 días 4.32s


Ridge ventana 7 días 4.03s

Ridge ventana 14 días 4.18s

Ridge ventana 21 días 4.18s

Ridge ventana 28 días 4.32s

In [ ]:
# SECCIÓN 6 ─ RIDGE MÉTRICAS
res_ridge = {v: metricas(y_true_ridge[v], y_pred_ridge[v]) for v in ventanas}
df_ridge = pd.DataFrame(res_ridge, index=["MAPE","MAE","RMSE","MSE","R2","LB_p"]).T
print(df_ridge)
print("Mejor ventana Ridge", df_ridge.RMSE.idxmin())
df_ridge


        MAPE       MAE      RMSE       MSE        R2      LB_p
7  1.571e+08 4.794e-01 7.274e-01 5.290e-01 6.937e-01 0.000e+00
14 1.530e+08 4.644e-01 7.183e-01 5.160e-01 7.014e-01 0.000e+00
21 1.518e+08 4.602e-01 8.000e-01 6.400e-01 6.296e-01 0.000e+00
28 1.519e+08 4.578e-01 7.828e-01 6.127e-01 6.454e-01 0.000e+00
Mejor ventana Ridge 14


,MAPE,MAE,RMSE,MSE,R2,LB_p
7,1.571e+08,4.794e-01,7.274e-01,5.290e-01,6.937e-01,0.000e+00
14,1.530e+08,4.644e-01,7.183e-01,5.160e-01,7.014e-01,0.000e+00
21,1.518e+08,4.602e-01,8.000e-01,6.400e-01,6.296e-01,0.000e+00
28,1.519e+08,4.578e-01,7.828e-01,6.127e-01,6.454e-01,0.000e+00


```
        MAPE       MAE      RMSE       MSE        R2      LB_p
7  1.571e+08 4.794e-01 7.274e-01 5.290e-01 6.937e-01 0.000e+00
14 1.530e+08 4.644e-01 7.183e-01 5.160e-01 7.014e-01 0.000e+00
21 1.518e+08 4.602e-01 8.000e-01 6.400e-01 6.296e-01 0.000e+00
28 1.519e+08 4.578e-01 7.828e-01 6.127e-01 6.454e-01 0.000e+00

Mejor ventana Ridge 14

MAPE	MAE	RMSE	MSE	R2	LB_p
7	1.571e+08	4.794e-01	7.274e-01	5.290e-01	6.937e-01	0.000e+00
14	1.530e+08	4.644e-01	7.183e-01	5.160e-01	7.014e-01	0.000e+00
21	1.518e+08	4.602e-01	8.000e-01	6.400e-01	6.296e-01	0.000e+00
28	1.519e+08	4.578e-01	7.828e-01	6.127e-01	6.454e-01	0.000e+00

```

  <div style="background-color: #f5fdf9; padding: 12px;">
    <p><strong>💡Análisis de los Resultados del Modelo Ridge</strong></p>
    <p>El modelo Ridge se corrio con diferentes tamaños de ventana (7, 14, 21, y 28).</p>
    <h4>1. Mejora en las Métricas con el Tamaño de la Ventana</h4>
    <strong>MAPE (Mean Absolute Percentage Error):</strong>
    <ul>
      <li>T=7: 1.571e+08</li>
      <li>T=14: 1.530e+08 (mejora del 2.6%)</li>
      <li>T=21: 1.518e+08 (mejora del 3.4% respecto a T=7)</li>
      <li>T=28: 1.519e+08 (similar a T=21)</li>
    </ul>
    <p>El MAPE sigue siendo muy elevado, lo cual indica un alto error porcentual en las predicciones.</p>
    <strong>MAE (Mean Absolute Error):</strong>
    <ul>
      <li>T=7: 4.794e-01</li>
      <li>T=14: 4.644e-01 (mejora del 3.2%)</li>
      <li>T=21: 4.602e-01 (mejora del 3.9% respecto a T=7)</li>
      <li>T=28: 4.578e-01 (mejora del 4.5% respecto a T=7)</li>
    </ul>
    <strong>RMSE (Root Mean Squared Error):</strong>
    <ul>
      <li>T=7: 7.274e-01</li>
      <li>T=14: 7.183e-01 (mejora del 1.3%)</li>
      <li>T=21: 8.000e-01 (empeora en un 10.3% respecto a T=14)</li>
      <li>T=28: 7.828e-01 (mejora del 2.2% respecto a T=21)</li>
    </ul>
    <strong>MSE (Mean Squared Error):</strong>
    <ul>
      <li>T=7: 5.290e-01</li>
      <li>T=14: 5.160e-01 (mejora del 2.5%)</li>
      <li>T=21: 6.400e-01 (empeora en un 23.8% respecto a T=14)</li>
      <li>T=28: 6.127e-01 (mejora del 4.3% respecto a T=21)</li>
    </ul>
    <strong>R2 (R-squared):</strong>
    <ul>
      <li>T=7: 6.937e-01 (R2=0.693)</li>
      <li>T=14: 7.014e-01 (R2=0.701, mejora del 1.5%)</li>
      <li>T=21: 6.296e-01 (R2=0.629, empeora en un 10.8% respecto a T=14)</li>
      <li>T=28: 6.454e-01 (R2=0.645, mejora del 2.5% respecto a T=21)</li>
    </ul>
    <h4>2. Mejor Ventana</h4>
    <p>La mejor ventana para el modelo Ridge es T=14, ya que presenta:</p>
    <ul>
      <li>El MAPE más bajo: 1.530e+08</li>
      <li>El RMSE más bajo: 7.183e-01</li>
      <li>El MSE más bajo: 5.160e-01</li>
      <li>El R2 más alto: 7.014e-01 (R2=0.701)</li>
    </ul>
    <h4>3. Observaciones</h4>
    <strong>MAPE Elevado:</strong>
    <p>Los valores de MAPE son extremadamente altos en todas las ventanas, lo que indica un error porcentual significativo en las predicciones. Esto podría darse por:</p>
    <ul>
      <li>Escala de las variables: Las variables explicativas o la variable objetivo podrían tener una escala muy grande.</li>
      <li>Ruido en los datos: La presencia de ruido o outliers podría estar afectando el rendimiento del modelo.</li>
    </ul>
    <strong>Autocorrelación en Residuos:</strong>
    <p>El valor LB_p es 0.e+00 en todas las ventanas, lo que indica autocorrelación significativa en los residuos. Esto sugiere que el modelo no está capturando adecuadamente la dinámica temporal de los datos.</p>
    <strong>Rendimiento Inconsistente:</strong>
    <p>Para T=21, se observa un empeoramiento en el RMSE y el MSE respecto a T=14, lo que podría indicar sobreajuste o la presencia de ruido en los datos para este tamaño de ventana.</p>
    <h4>4. Comparación con Otros Modelos</h4>
    <strong>Modelo de Regresión Lineal:</strong>
    <p>En comparación con los resultados de la Regresión Lineal, el modelo Ridge presenta un mejor R2 y menores valores de MAE y RMSE para la mayoría de las ventanas. Sin embargo, el MAPE sigue siendo muy elevado en ambos modelos.</p>
    <strong>Modelo KNN:</strong>
    <p>El modelo Ridge muestra un mejor desempeño en términos de MAE, RMSE y MSE en casi todas las ventanas, y un R2 comparable o superior.</p>
    <h4>5. Conclusión</h4>
    <p>El modelo Ridge muestra un rendimiento superior en comparación con los otros modelos evaluados, especialmente para la ventana T=14. Sin embargo, los altos valores de MAPE y la autocorrelación en los residuos sugieren que el modelo aún no está capturando adecuadamente la dinámica temporal de los datos.</p>
  </div>
</div>


### Codigo de Lasso

In [ ]:
# SECCIÓN 7 ─ LASSO ENTRENAMIENTO
modelo_lasso = Lasso(max_iter=500)
y_true_lasso, y_pred_lasso = {}, {}
for v in ventanas:
    t0 = time.time()
    yt, yp = obtener_predicciones(modelo_lasso, v)
    y_true_lasso[v], y_pred_lasso[v] = yt, yp
    print(f"Lasso ventana {v} días {time.time()-t0:.2f}s")


Lasso ventana 7 días 4.25s
Lasso ventana 14 días 4.40s
Lasso ventana 21 días 4.38s
Lasso ventana 28 días 4.49s


Lasso ventana 7 días 4.25s

Lasso ventana 14 días 4.40s

Lasso ventana 21 días 4.38s

Lasso ventana 28 días 4.49s

In [ ]:
# SECCIÓN 8 ─ LASSO MÉTRICAS
res_lasso = {v: metricas(y_true_lasso[v], y_pred_lasso[v]) for v in ventanas}
df_lasso = pd.DataFrame(res_lasso, index=["MAPE","MAE","RMSE","MSE","R2","LB_p"]).T
print(df_lasso)
print("Mejor ventana Lasso", df_lasso.RMSE.idxmin())
df_lasso


        MAPE       MAE      RMSE       MSE        R2    LB_p
7  3.530e+08 7.098e-01 9.081e-01 8.247e-01 5.225e-01  0.e+00
14 3.441e+08 6.654e-01 8.482e-01 7.195e-01 5.836e-01  0.e+00
21 3.387e+08 6.476e-01 8.254e-01 6.812e-01 6.057e-01  0.e+00
28 3.356e+08 6.392e-01 8.142e-01 6.629e-01 6.165e-01  0.e+00
Mejor ventana Lasso 28


,MAPE,MAE,RMSE,MSE,R2,LB_p
7,3.530e+08,7.098e-01,9.081e-01,8.247e-01,5.225e-01,0.e+00
14,3.441e+08,6.654e-01,8.482e-01,7.195e-01,5.836e-01,0.e+00
21,3.387e+08,6.476e-01,8.254e-01,6.812e-01,6.057e-01,0.e+00
28,3.356e+08,6.392e-01,8.142e-01,6.629e-01,6.165e-01,0.e+00


```
        MAPE       MAE      RMSE       MSE        R2    LB_p
7  3.530e+08 7.098e-01 9.081e-01 8.247e-01 5.225e-01  0.e+00
14 3.441e+08 6.654e-01 8.482e-01 7.195e-01 5.836e-01  0.e+00
21 3.387e+08 6.476e-01 8.254e-01 6.812e-01 6.057e-01  0.e+00
28 3.356e+08 6.392e-01 8.142e-01 6.629e-01 6.165e-01  0.e+00

Mejor ventana Lasso 28

MAPE	MAE	RMSE	MSE	R2	LB_p
7	3.530e+08	7.098e-01	9.081e-01	8.247e-01	5.225e-01	0.e+00
14	3.441e+08	6.654e-01	8.482e-01	7.195e-01	5.836e-01	0.e+00
21	3.387e+08	6.476e-01	8.254e-01	6.812e-01	6.057e-01	0.e+00
28	3.356e+08	6.392e-01	8.142e-01	6.629e-01	6.165e-01	0.e+00

```

<div style="background-color: #f5fdf9; padding: 12px;"> <h3><strong>💡Análisis del Modelo Lasso</strong></h3> <p>A continuación, se presenta el análisis de los resultados obtenidos con el modelo Lasso para diferentes tamaños de ventana (T = 7, 28).</p> <hr> <h3><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h3> <h4><strong>1.1. Reducción del Error</strong></h4> <ul> <li><strong>MAPE (Mean Absolute Percentage Error):</strong> El MAPE disminuye ligeramente con el aumento del tamaño de la ventana: <ul> <li>T=7: <code>3.530e+08</code></li> <li>T=28: <code>3.356e+08</code> (reducción del 4.9%)</li> </ul> </li> <li><strong>MAE (Mean Absolute Error):</strong> El error absoluto también disminuye: <ul> <li>T=7: <code>7.098e-01</code></li> <li>T=28: <code>6.392e-01</code> (reducción del 9.9%)</li> </ul> </li> <li><strong>RMSE (Root Mean Squared Error):</strong> El error cuadrático medio muestra una mejora: <ul> <li>T=7: <code>9.081e-01</code></li> <li>T=28: <code>8.142e-01</code> (reducción del 10.3%)</li> </ul> </li> <li><strong>MSE (Mean Squared Error):</strong> El error cuadrático medio sigue una tendencia similar: <ul> <li>T=7: <code>8.247e-01</code></li> <li>T=28: <code>6.629e-01</code> (reducción del 19.8%)</li> </ul> </li> </ul> <h4><strong>1.2. Mejora en el Coeficiente de Determinación (R2)</strong></h4> <ul> <li><strong>R2:</strong> El coeficiente de determinación aumenta con el tamaño de la ventana: <ul> <li>T=7: <code>5.225e-01</code> (R2=0.522)</li> <li>T=28: <code>6.165e-01</code> (R2=0.616)</li> </ul> </li> </ul> <hr> <h3><strong>2. Ventana Óptima</strong></h3> <p>La ventana de tamaño <strong>T=28</strong> muestra los mejores resultados:</p> <ul> <li><strong>MAPE más bajo:</strong> <code>3.356e+08</code></li> <li><strong>RMSE más bajo:</strong> <code>8.142e-01</code></li> <li><strong>MSE más bajo:</strong> <code>6.629e-01</code></li> <li><strong>R2 más alto:</strong> <code>6.165e-01</code></li> </ul> <hr> <h3><strong>3. Observaciones</strong></h3> <ul> <li><strong>MAPE Elevado:</strong> Los valores de MAPE siguen siendo muy altos, lo que podría deberse a: <ul> <li>Escala de las variables.</li> </li> <li>Limitaciones del modelo Lasso para capturar relaciones no lineales.</li> </ul> </li> <li><strong>Autocorrelación en los residuos:</strong> El valor LB_p es 0.e+00 en todas las ventanas, indicando que el modelo no captura adecuadamente la dinámica temporal.</li> </ul> <hr> <h3> <h3><strong>Conclusión</strong></h3> <p>El modelo Lasso mejora sus métricas con ventanas mayores, alcanzando su mejor desempeño en T=28. Sin embargo, presenta limitaciones importantes, especialmente en el MAPE y la autocorrelación de los residuos, lo que sugiere la necesidad de ajustes adicionales o el uso de modelos más complejos para capturar la dinámica de los datos.</p> </div>

### Codigo de Arbol de decision

In [ ]:
# SECCIÓN 9 ─ ÁRBOL DE DECISIÓN ENTRENAMIENTO
modelo_dt = DecisionTreeRegressor(max_depth=5, random_state=0)
y_true_dt, y_pred_dt = {}, {}
for v in ventanas:
    t0 = time.time()
    yt, yp = obtener_predicciones(modelo_dt, v)
    y_true_dt[v], y_pred_dt[v] = yt, yp
    print(f"DT ventana {v} días {time.time()-t0:.2f}s")


DT ventana 7 días 5.83s
DT ventana 14 días 7.71s
DT ventana 21 días 9.49s
DT ventana 28 días 11.12s


DT ventana 7 días 5.83s

DT ventana 14 días 7.71s

DT ventana 21 días 9.49s

DT ventana 28 días 11.12s

In [ ]:
# SECCIÓN 10 ─ ÁRBOL DE DECISIÓN MÉTRICAS
res_dt = {v: metricas(y_true_dt[v], y_pred_dt[v]) for v in ventanas}
df_dt = pd.DataFrame(res_dt, index=["MAPE","MAE","RMSE","MSE","R2","LB_p"]).T
print(df_dt)
print("Mejor ventana DT", df_dt.RMSE.idxmin())
df_dt


        MAPE       MAE      RMSE       MSE        R2    LB_p
7  1.740e+08 6.015e-01 8.064e-01 6.503e-01 6.235e-01  0.e+00
14 1.537e+08 5.500e-01 7.399e-01 5.475e-01 6.832e-01  0.e+00
21 1.552e+08 5.283e-01 7.117e-01 5.065e-01 7.068e-01  0.e+00
28 1.489e+08 5.145e-01 6.918e-01 4.786e-01 7.231e-01  0.e+00
Mejor ventana DT 28


,MAPE,MAE,RMSE,MSE,R2,LB_p
7,1.740e+08,6.015e-01,8.064e-01,6.503e-01,6.235e-01,0.e+00
14,1.537e+08,5.500e-01,7.399e-01,5.475e-01,6.832e-01,0.e+00
21,1.552e+08,5.283e-01,7.117e-01,5.065e-01,7.068e-01,0.e+00
28,1.489e+08,5.145e-01,6.918e-01,4.786e-01,7.231e-01,0.e+00


```
        MAPE       MAE      RMSE       MSE        R2    LB_p
7  1.740e+08 6.015e-01 8.064e-01 6.503e-01 6.235e-01  0.e+00
14 1.537e+08 5.500e-01 7.399e-01 5.475e-01 6.832e-01  0.e+00
21 1.552e+08 5.283e-01 7.117e-01 5.065e-01 7.068e-01  0.e+00
28 1.489e+08 5.145e-01 6.918e-01 4.786e-01 7.231e-01  0.e+00

Mejor ventana DT 28

MAPE	MAE	RMSE	MSE	R2	LB_p
7	1.740e+08	6.015e-01	8.064e-01	6.503e-01	6.235e-01	0.e+00
14	1.537e+08	5.500e-01	7.399e-01	5.475e-01	6.832e-01	0.e+00
21	1.552e+08	5.283e-01	7.117e-01	5.065e-01	7.068e-01	0.e+00
28	1.489e+08	5.145e-01	6.918e-01	4.786e-01	7.231e-01	0.e+00
```

<div style="background-color: #f5fdf9; padding: 12px;"> <h3><strong>💡Análisis del Modelo de Árbol de Decisión</strong></h3> <p>A continuación, se presenta el análisis de los resultados obtenidos con el modelo de Árbol de Decisión para diferentes tamaños de ventana (T = 7, 28).</p> <hr> <h3><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h3> <h4><strong>1.1. Reducción del Error</strong></h4> <ul> <li><strong>MAPE (Mean Absolute Percentage Error):</strong> El MAPE disminuye con el aumento del tamaño de la ventana: <ul> <li>T=7: <code>1.740e+08</code></li> <li>T=28: <code>1.489e+08</code> (reducción del 14.5%)</li> </ul> </li> <li><strong>MAE (Mean Absolute Error):</strong> El error absoluto también disminuye: <ul> <li>T=7: <code>6.015e-01</code></li> <li>T=28: <code>5.145e-01</code> (reducción del 14.5%)</li> </ul> </li> <li><strong>RMSE (Root Mean Squared Error):</strong> El error cuadrático medio muestra una mejora: <ul> <li>T=7: <code>8.064e-01</code></li> <li>T=28: <code>6.918e-01</code> (reducción del 14.2%)</li> </ul> </li> <li><strong>MSE (Mean Squared Error):</strong> El error cuadrático medio sigue una tendencia similar: <ul> <li>T=7: <code>6.503e-01</code></li> <li>T=28: <code>4.786e-01</code> (reducción del 26.4%)</li> </ul> </li> </ul> <h4><strong>1.2. Mejora en el Coeficiente de Determinación (R2)</strong></h4> <ul> <li><strong>R2:</strong> El coeficiente de determinación mejora con el tamaño de la ventana: <ul> <li>T=7: <code>6.235e-01</code> (R2=0.623)</li> <li>T=28: <code>7.231e-01</code> (R2=0.723)</li> </ul> </li> </ul> <hr> <h3><strong>2. Mejor Ventana</strong></h3> <p>La ventana de tamaño <strong>T=28</strong> muestra los mejores resultados:</p> <ul> <li><strong>MAPE más bajo:</strong> <code>1.489e+08</code></li> <li><strong>MAE más bajo:</strong> <code>5.145e-01</code></li> <li><strong>RMSE más bajo:</strong> <code>6.918e-01</code></li> <li><strong>MSE más bajo:</strong> <code>4.786e-01</code></li> <li><strong>R2 más alto:</strong> <code>7.231e-01</code></li> </ul> <hr> <h3><strong>3. Observaciones</strong></h3> <ul> <li><strong>MAPE Elevado:</strong> Aunque mejora con ventanas más grandes, sigue siendo alto debido posiblemente a: <ul> <li>Relaciones no lineales no capturadas completamente.</li></li> </ul> </li> <li><strong>Autocorrelación en Residuos:</strong> El valor <code>LB_p=0.e+00</code> en todas las ventanas indica fuerte autocorrelación, reflejando que el modelo no captura bien la dinámica temporal.</li> <li><strong>Sobreajuste Potencial:</strong> La mejora al aumentar el tamaño de la ventana puede estar asociada a una mejor generalización del modelo ante más datos.</li> </ul> <hr> <h3><strong>4. Comparación con los anteriores Modelos</strong></h3> <ul> <li><strong>Modelo Lasso:</strong> El Árbol de Decisión supera ampliamente al Lasso, especialmente en R2.</li> <li><strong>Modelo Ridge:</strong> Mejor R2 en ventanas grandes comparado con Ridge.</li> <li><strong>Modelo KNN:</strong> Mejor R2 y menores errores en todas las ventanas.</li> <li><strong>Regresión Lineal:</strong> Supera ampliamente en R2 y errores a la Regresión Lineal.</li> </ul> <hr> <h3><strong>Conclusión</strong></h3> <p>El modelo de Árbol de Decisión muestra mejoras claras con ventanas más grandes, siendo <strong>T=28</strong> el mejor escenario. No obstante, los altos valores de MAPE y la autocorrelación en los residuos indican la necesidad de explorar métodos más robustos y complejos para mejorar el desempeño predictivo.</p> </div>

### Codigo de Random Forest

In [ ]:
# SECCIÓN 11 ─ RANDOM FOREST ENTRENAMIENTO
modelo_rf = RandomForestRegressor(n_estimators=20, random_state=0, n_jobs=-1)
y_true_rf, y_pred_rf = {}, {}
for v in ventanas:
    t0 = time.time()
    yt, yp = obtener_predicciones(modelo_rf, v)
    y_true_rf[v], y_pred_rf[v] = yt, yp
    print(f"RF ventana {v} días {time.time()-t0:.2f}s")


RF ventana 7 días 173.92s
RF ventana 14 días 185.76s
RF ventana 21 días 201.95s
RF ventana 28 días 206.32s


RF ventana 7 días 173.92s

RF ventana 14 días 185.76s

RF ventana 21 días 201.95s

RF ventana 28 días 206.32s

In [ ]:
# SECCIÓN 12 ─ RANDOM FOREST MÉTRICAS
res_rf = {v: metricas(y_true_rf[v], y_pred_rf[v]) for v in ventanas}
df_rf = pd.DataFrame(res_rf, index=["MAPE","MAE","RMSE","MSE","R2","LB_p"]).T
print(df_rf)
print("Mejor ventana RF", df_rf.RMSE.idxmin())
df_rf

        MAPE       MAE      RMSE       MSE        R2    LB_p
7  1.816e+08 5.088e-01 6.694e-01 4.481e-01 7.405e-01  0.e+00
14 1.616e+08 4.877e-01 6.418e-01 4.119e-01 7.617e-01  0.e+00
21 1.565e+08 4.803e-01 6.339e-01 4.018e-01 7.674e-01  0.e+00
28 1.514e+08 4.763e-01 6.280e-01 3.944e-01 7.718e-01  0.e+00
Mejor ventana RF 28


,MAPE,MAE,RMSE,MSE,R2,LB_p
7,1.816e+08,5.088e-01,6.694e-01,4.481e-01,7.405e-01,0.e+00
14,1.616e+08,4.877e-01,6.418e-01,4.119e-01,7.617e-01,0.e+00
21,1.565e+08,4.803e-01,6.339e-01,4.018e-01,7.674e-01,0.e+00
28,1.514e+08,4.763e-01,6.280e-01,3.944e-01,7.718e-01,0.e+00


```
        MAPE       MAE      RMSE       MSE        R2    LB_p
7  1.816e+08 5.088e-01 6.694e-01 4.481e-01 7.405e-01  0.e+00
14 1.616e+08 4.877e-01 6.418e-01 4.119e-01 7.617e-01  0.e+00
21 1.565e+08 4.803e-01 6.339e-01 4.018e-01 7.674e-01  0.e+00
28 1.514e+08 4.763e-01 6.280e-01 3.944e-01 7.718e-01  0.e+00

Mejor ventana RF 28

MAPE	MAE	RMSE	MSE	R2	LB_p
7	1.816e+08	5.088e-01	6.694e-01	4.481e-01	7.405e-01	0.e+00
14	1.616e+08	4.877e-01	6.418e-01	4.119e-01	7.617e-01	0.e+00
21	1.565e+08	4.803e-01	6.339e-01	4.018e-01	7.674e-01	0.e+00
28	1.514e+08	4.763e-01	6.280e-01	3.944e-01	7.718e-01	0.e+00
```

<div style="background-color: #f5fdf9; padding: 12px;"> <h3><strong>💡Análisis de los Resultados del Modelo Random Forest</strong></h3> <p>A continuación, se realiza el análisis de los resultados obtenidos con el modelo Random Forest, utilizando diferentes tamaños de ventana (T = 7, 14, 21, 28).</p> <hr> <h3><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h3> <h4><strong>1.1. Reducción del Error</strong></h4> <ul> <li><strong>MAPE (Mean Absolute Percentage Error):</strong> El MAPE disminuye gradualmente con el aumento del tamaño de la ventana: <ul> <li>T=7: <code>1.816e+08</code></li> <li>T=28: <code>1.514e+08</code> (reducción del 16.6%)</li> </ul> </li> <li><strong>MAE (Mean Absolute Error):</strong> El error absoluto también disminuye con el aumento del tamaño de la ventana: <ul> <li>T=7: <code>5.088e-01</code></li> <li>T=28: <code>4.763e-01</code> (reducción del 6.3%)</li> </ul> </li> <li><strong>RMSE (Root Mean Squared Error):</strong> El error cuadrático medio muestra una mejora constante: <ul> <li>T=7: <code>6.694e-01</code></li> <li>T=28: <code>6.280e-01</code> (reducción del 6.1%)</li> </ul> </li> <li><strong>MSE (Mean Squared Error):</strong> El error cuadrático medio sigue una tendencia similar al RMSE: <ul> <li>T=7: <code>4.481e-01</code></li> <li>T=28: <code>3.944e-01</code> (reducción del 12.0%)</li> </ul> </li> </ul> <h4><strong>1.2. Mejora en el Coeficiente de Determinación (R2)</strong></h4> <ul> <li><strong>R2:</strong> El coeficiente de determinación mejora con el tamaño de la ventana: <ul> <li>T=7: <code>7.405e-01</code> (R2=0.740)</li> <li>T=28: <code>7.718e-01</code> (R2=0.772)</li> </ul> </li> </ul> <hr> <h3><strong>2. Mejor Ventana</strong></h3> <p>La mejor ventana para el modelo Random Forest es <strong>T=28</strong>, ya que presenta:</p> <ul> <li><strong>MAPE más bajo:</strong> <code>1.514e+08</code></li> <li><strong>MAE más bajo:</strong> <code>4.763e-01</code></li> <li><strong>RMSE más bajo:</strong> <code>6.280e-01</code></li> <li><strong>MSE más bajo:</strong> <code>3.944e-01</code></li> <li><strong>R2 más alto:</strong> <code>7.718e-01</code></li> </ul> <hr> <h3><strong>3. Observaciones</strong></h3> <ul> <li><strong>MAPE Elevado:</strong> Aunque mejora con ventanas más grandes, los valores siguen siendo muy altos debido a: <ul> <li>Escala de las variables o de la variable objetivo.</li> </li> </ul> </li> <li><strong>Autocorrelación en Residuos:</strong> El valor <code>LB_p=0.e+00</code> en todas las ventanas indica autocorrelación significativa, reflejando que el modelo no captura adecuadamente la dinámica temporal.</li> </li> </ul> <hr> <h3><strong>4. Comparación con los Otros Modelos</strong></h3> <ul> <li><strong>Modelo de Árbol de Decisión:</strong> Random Forest supera al Árbol de Decisión en todas las métricas, especialmente en R2.</li> <li><strong>Modelo Lasso:</strong> Random Forest ofrece un mejor rendimiento en MAPE, MAE, RMSE, MSE y R2.</li> <li><strong>Modelo KNN:</strong> Random Forest muestra un mejor R2 y menores errores (MAE, RMSE) en todas las ventanas.</li> <li><strong>Modelo Ridge:</strong> En ventanas pequeñas (T=7 y T=14), Random Forest tiene un mejor R2 y menores errores que Ridge.</li> </ul> <hr> <h3><strong>Conclusión</strong></h3> <p>El modelo Random Forest muestra un rendimiento mejorado con ventanas más grandes, alcanzando su mejor desempeño en <strong>T=28</strong>. Sin embargo, los valores elevados de MAPE y la autocorrelación en los residuos sugieren que se necesita un enfoque más robusto para manejar la complejidad de los datos. Se recomienda explorar modelos más complejos, ajustar hiperparámetros y añadir características temporales para mejorar las predicciones.</p> </div>

### Codigo de XGBoost

In [ ]:
# SECCIÓN 13 ─ XGBOOST ENTRENAMIENTO
modelo_xgb = XGBRegressor(
    n_estimators=50,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    n_jobs=-1,
    random_state=0
)
y_true_xgb, y_pred_xgb = {}, {}
for v in ventanas:
    t0 = time.time()
    yt, yp = obtener_predicciones(modelo_xgb, v)
    y_true_xgb[v], y_pred_xgb[v] = yt, yp
    print(f"XGB ventana {v} días {time.time()-t0:.2f}s")


XGB ventana 7 días 76.78s
XGB ventana 14 días 81.64s
XGB ventana 21 días 83.22s
XGB ventana 28 días 88.77s


XGB ventana 7 días 76.78s

XGB ventana 14 días 81.64s

XGB ventana 21 días 83.22s

XGB ventana 28 días 88.77s

In [ ]:
# SECCIÓN 14 ─ XGBOOST MÉTRICAS
res_xgb = {v: metricas(y_true_xgb[v], y_pred_xgb[v]) for v in ventanas}
df_xgb = pd.DataFrame(res_xgb, index=["MAPE","MAE","RMSE","MSE","R2","LB_p"]).T
print(df_xgb)
print("Mejor ventana XGB", df_xgb.RMSE.idxmin())
df_xgb


        MAPE       MAE      RMSE       MSE        R2    LB_p
7  2.384e+08 5.554e-01 7.285e-01 5.307e-01 6.927e-01  0.e+00
14 2.098e+08 5.135e-01 6.747e-01 4.553e-01 7.365e-01  0.e+00
21 1.982e+08 4.937e-01 6.497e-01 4.221e-01 7.557e-01  0.e+00
28 1.903e+08 4.831e-01 6.360e-01 4.044e-01 7.660e-01  0.e+00
Mejor ventana XGB 28


,MAPE,MAE,RMSE,MSE,R2,LB_p
7,2.384e+08,5.554e-01,7.285e-01,5.307e-01,6.927e-01,0.e+00
14,2.098e+08,5.135e-01,6.747e-01,4.553e-01,7.365e-01,0.e+00
21,1.982e+08,4.937e-01,6.497e-01,4.221e-01,7.557e-01,0.e+00
28,1.903e+08,4.831e-01,6.360e-01,4.044e-01,7.660e-01,0.e+00


```
        MAPE       MAE      RMSE       MSE        R2    LB_p
7  2.384e+08 5.554e-01 7.285e-01 5.307e-01 6.927e-01  0.e+00
14 2.098e+08 5.135e-01 6.747e-01 4.553e-01 7.365e-01  0.e+00
21 1.982e+08 4.937e-01 6.497e-01 4.221e-01 7.557e-01  0.e+00
28 1.903e+08 4.831e-01 6.360e-01 4.044e-01 7.660e-01  0.e+00

Mejor ventana XGB 28

MAPE	MAE	RMSE	MSE	R2	LB_p
7	2.384e+08	5.554e-01	7.285e-01	5.307e-01	6.927e-01	0.e+00
14	2.098e+08	5.135e-01	6.747e-01	4.553e-01	7.365e-01	0.e+00
21	1.982e+08	4.937e-01	6.497e-01	4.221e-01	7.557e-01	0.e+00
28	1.903e+08	4.831e-01	6.360e-01	4.044e-01	7.660e-01	0.e+00

```


<div style="background-color: #f5fdf9; padding: 12px;">
  <h3><strong>💡 Análisis del Modelo XGBoost</strong></h3>
  <p>Los resultados del modelo XGBoost para diferentes tamaños de ventana (T = 7, 14, 21, 28) muestran una mejora constante en las métricas a medida que aumenta el tamaño de la ventana.</p>
  <hr>
  <h3><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h3>
  <h4><strong>1.1. Reducción del Error</strong></h4>
  <ul>
    <li><strong>MAPE (Mean Absolute Percentage Error):</strong> El MAPE disminuye significativamente con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>2.384e+08</code></li>
        <li>T=28: <code>1.903e+08</code> (reducción del 20.2%)</li>
      </ul>
    </li>
    <li><strong>MAE (Mean Absolute Error):</strong> El error absoluto disminuye con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>5.554e-01</code></li>
        <li>T=28: <code>4.831e-01</code> (reducción del 13.1%)</li>
      </ul>
    </li>
    <li><strong>RMSE (Root Mean Squared Error):</strong> El error cuadrático medio muestra una mejora constante:
      <ul>
        <li>T=7: <code>7.285e-01</code></li>
        <li>T=28: <code>6.360e-01</code> (reducción del 12.7%)</li>
      </ul>
    </li>
    <li><strong>MSE (Mean Squared Error):</strong> El error cuadrático medio sigue una tendencia similar al RMSE:
      <ul>
        <li>T=7: <code>5.307e-01</code></li>
        <li>T=28: <code>4.044e-01</code> (reducción del 23.9%)</li>
      </ul>
    </li>
  </ul>
  <h4><strong>1.2. Mejora en el Coeficiente de Determinación (R2)</strong></h4>
  <ul>
    <li><strong>R2:</strong> El coeficiente de determinación mejora con el tamaño de la ventana:
      <ul>
        <li>T=7: <code>6.927e-01</code> (R2=0.692)</li>
        <li>T=28: <code>7.660e-01</code> (R2=0.766)</li>
      </ul>
    </li>
  </ul>
  <hr>
  <h3><strong>2. Mejor Ventana</strong></h3>
  <p>La mejor ventana para el modelo XGBoost es <strong>T=28</strong>, ya que presenta:</p>
  <ul>
     <li><strong>MAPE más bajo:</strong> <code>1.903e+08</code></li>
     <li><strong>MAE más bajo:</strong> <code>4.831e-01</code></li>
     <li><strong>RMSE más bajo:</strong> <code>6.360e-01</code></li>
     <li><strong>MSE más bajo:</strong> <code>4.044e-01</code></li>
     <li><strong>R2 más alto:</strong> <code>7.660e-01</code></li>
  </ul>
  <hr>
  <h3><strong>3. Observaciones</strong></h3>
  <ul>
     <li><strong>MAPE Elevado:</strong> Aunque el MAPE mejora con ventanas más grandes, los valores siguen siendo altos, lo que indica un alto error porcentual en las predicciones. Esto podría deberse a:
      <ul>
        <li>Relaciones complejas que el modelo XGBoost no captura completamente.
     </ul>
     </li>
     <li><strong>Autocorrelación en Residuos:</strong> El valor <code>LB_p=0.e+00</code> en todas las ventanas indica autocorrelación significativa en los residuos, sugiriendo que el modelo no captura adecuadamente la dinámica temporal.</li>
  </ul>
  <hr>
  <h3><strong>4. Comparación con los Otros Modelos</strong></h3>
  <ul>
    <li><strong>Modelo Random Forest:</strong> XGBoost supera a Random Forest en todas las métricas, especialmente en R2.</li>
    <li><strong>Modelo de Árbol de Decisión:</strong> XGBoost ofrece un mejor rendimiento en MAPE, MAE, RMSE, MSE y R2.</li>
    <li><strong>Modelo Ridge:</strong> En ventanas más pequeñas (T=7 y T=14), XGBoost tiene un mejor R2 y menores errores que Ridge.</li>
    <li><strong>Modelo KNN:</strong> XGBoost muestra un mejor R2 y menores errores (MAE, RMSE) en todas las ventanas.</li>
  </ul>
  <hr>
  
  <h3><strong>Conclusión</strong></h3>
  <p>El modelo XGBoost muestra un rendimiento superior en comparación con otros modelos evaluados, alcanzando su mejor desempeño en <strong>T=28</strong>. Sin embargo, los altos valores de MAPE y la autocorrelación en los residuos sugieren que se necesita un enfoque más robusto para manejar la complejidad de los datos. </p>
</div>

### Codigo de SVM

In [ ]:
# SECCIÓN 15 ─ SVM ENTRENAMIENTO
modelo_svm = Pipeline([("esc", StandardScaler()), ("svm", SVR(kernel="linear"))])
y_true_svm, y_pred_svm = {}, {}
for v in ventanas:
    t0 = time.time()
    yt, yp = obtener_predicciones(modelo_svm, v)
    y_true_svm[v], y_pred_svm[v] = yt, yp
    print(f"SVM ventana {v} días {time.time()-t0:.2f}s")


SVM ventana 7 días 26.19s
SVM ventana 14 días 72.21s
SVM ventana 21 días 146.09s
SVM ventana 28 días 234.86s


SVM ventana 7 días 26.19s

SVM ventana 14 días 72.21s

SVM ventana 21 días 146.09s

SVM ventana 28 días 234.86s

In [ ]:
# SECCIÓN 16 ─ SVM MÉTRICAS
res_svm = {v: metricas(y_true_svm[v], y_pred_svm[v]) for v in ventanas}
df_svm = pd.DataFrame(res_svm, index=["MAPE","MAE","RMSE","MSE","R2","LB_p"]).T
print(df_svm)
print("Mejor ventana SVM", df_svm.RMSE.idxmin())
df_svm


        MAPE       MAE      RMSE       MSE        R2    LB_p
7  1.686e+08 4.860e-01 1.107e+00 1.226e+00 2.902e-01  0.e+00
14 1.606e+08 4.643e-01 9.407e-01 8.849e-01 4.879e-01  0.e+00
21 1.584e+08 4.533e-01 7.410e-01 5.491e-01 6.822e-01  0.e+00
28 1.577e+08 4.515e-01 7.305e-01 5.337e-01 6.912e-01  0.e+00
Mejor ventana SVM 28


,MAPE,MAE,RMSE,MSE,R2,LB_p
7,1.686e+08,4.860e-01,1.107e+00,1.226e+00,2.902e-01,0.e+00
14,1.606e+08,4.643e-01,9.407e-01,8.849e-01,4.879e-01,0.e+00
21,1.584e+08,4.533e-01,7.410e-01,5.491e-01,6.822e-01,0.e+00
28,1.577e+08,4.515e-01,7.305e-01,5.337e-01,6.912e-01,0.e+00


```
        MAPE       MAE      RMSE       MSE        R2    LB_p
7  1.686e+08 4.860e-01 1.107e+00 1.226e+00 2.902e-01  0.e+00
14 1.606e+08 4.643e-01 9.407e-01 8.849e-01 4.879e-01  0.e+00
21 1.584e+08 4.533e-01 7.410e-01 5.491e-01 6.822e-01  0.e+00
28 1.577e+08 4.515e-01 7.305e-01 5.337e-01 6.912e-01  0.e+00

Mejor ventana SVM 28

MAPE	MAE	RMSE	MSE	R2	LB_p
7	1.686e+08	4.860e-01	1.107e+00	1.226e+00	2.902e-01	0.e+00
14	1.606e+08	4.643e-01	9.407e-01	8.849e-01	4.879e-01	0.e+00
21	1.584e+08	4.533e-01	7.410e-01	5.491e-01	6.822e-01	0.e+00
28	1.577e+08	4.515e-01	7.305e-01	5.337e-01	6.912e-01	0.e+00

```

<div style="background-color: #f5fdf9; padding: 12px;">
  <h3><strong>💡 Análisis del Modelo SVM</strong></h3>
  <p>A continuación, se presenta el análisis de los resultados obtenidos con el modelo SVM (Máquinas de Soporte Vectorial), utilizando diferentes tamaños de ventana (T = 7, 14, 21, 28).</p>
  <hr>
  <h3><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h3>
  <h4><strong>1.1. Reducción del Error</strong></h4>
  <ul>
    <li><strong>MAPE (Mean Absolute Percentage Error):</strong> El MAPE disminuye ligeramente con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>1.686e+08</code></li>
        <li>T=28: <code>1.577e+08</code> (reducción del 6.5%)</li>
      </ul>
    </li>
    <li><strong>MAE (Mean Absolute Error):</strong> El error absoluto también disminuye ligeramente con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>4.860e-01</code></li>
        <li>T=28: <code>4.515e-01</code> (reducción del 7.2%)</li>
      </ul>
    </li>
    <li><strong>RMSE (Root Mean Squared Error):</strong> El error cuadrático medio muestra una mejora más notable:
      <ul>
        <li>T=7: <code>1.107e+00</code></li>
        <li>T=28: <code>7.305e-01</code> (reducción del 34.1%)</li>
      </ul>
    </li>
    <li><strong>MSE (Mean Squared Error):</strong> El error cuadrático medio sigue una tendencia similar al RMSE:
      <ul>
        <li>T=7: <code>1.226e+00</code></li>
        <li>T=28: <code>5.337e-01</code> (reducción del 56.5%)</li>
      </ul>
    </li>
  </ul>
  <h4><strong>1.2. Mejora en el Coeficiente de Determinación (R2)</strong></h4>
  <ul>
    <li><strong>R2:</strong> El coeficiente de determinación mejora significativamente con el tamaño de la ventana:
      <ul>
        <li>T=7: <code>2.902e-01</code> (R2=0.290)</li>
        <li>T=28: <code>6.912e-01</code> (R2=0.691)</li>
      </ul>
    </li>
  </ul>
  <hr>
  <h3><strong>2. Mejor Ventana</strong></h3>
  <p>La mejor ventana para el modelo SVM es <strong>T=28</strong>, ya que presenta:</p>
  <ul>
    <li><strong>MAPE más bajo:</strong> <code>1.577e+08</code></li>
    <li><strong>MAE más bajo:</strong> <code>4.515e-01</code></li>
    <li><strong>RMSE más bajo:</strong> <code>7.305e-01</code></li>
    <li><strong>MSE más bajo:</strong> <code>5.337e-01</code></li>
    <li><strong>R2 más alto:</strong> <code>6.912e-01</code></li>
  </ul>
  <hr>
  <h3><strong>3. Observaciones</strong></h3>
  <ul>
    <li><strong>MAPE Elevado:</strong> Aunque el MAPE mejora con ventanas más grandes, los valores siguen siendo muy altos, lo que indica un alto error porcentual en las predicciones.
      <ul>
      </ul>
    </li>
    <li><strong>Autocorrelación en Residuos:</strong> El valor <code>LB_p=0.e+00</code> en todas las ventanas indica autocorrelación significativa en los residuos, sugiriendo que el modelo no está capturando adecuadamente la dinámica temporal de los datos.</li>
    <li><strong>Complejidad del Modelo:</strong> El SVM puede estar limitado en su capacidad para capturar relaciones complejas, especialmente si se utiliza un kernel lineal. Un kernel no lineal podría mejorar el rendimiento pero también aumentaría la complejidad computacional.</li>
  </ul>
  <hr>
  <h3><strong>4. Comparación con los Otros Modelos</strong></h3>
  <ul>
    <li><strong>Modelo XGBoost:</strong> SVM tiene un desempeño inferior en términos de MAPE, MAE y R2 en todas las ventanas.</li>
    <li><strong>Modelo Random Forest:</strong> SVM ofrece resultados inferiores a los de Random Forest, especialmente en R2.</li>
    <li><strong>Modelo Ridge:</strong> En ventanas más pequeñas, SVM tiene un R2 inferior al de Ridge, pero en ventanas más grandes, SVM alcanza un R2 similar.</li>
    <li><strong>Modelo Lasso:</strong> SVM supera a Lasso en términos de R2 y errores en ventanas más grandes.</li>
  </ul>
  <hr>
  <h3>
  <h3><strong>Conclusión</strong></h3>
  <p>El modelo SVM muestra una mejora en las métricas con ventanas más grandes, alcanzando su mejor desempeño en <strong>T=28</strong>. Sin embargo, su rendimiento sigue siendo inferior al de modelos como Random Forest y XGBoost. Los altos valores de MAPE y la autocorrelación en los residuos sugieren que se necesita un enfoque más robusto para manejar la complejidad de los datos.</p>
</div>

### Codigo de MLP

In [ ]:
# SECCIÓN 17 ─ MLP ENTRENAMIENTO
modelo_mlp = Pipeline([
    ("esc", StandardScaler()),
    ("mlp", MLPRegressor(hidden_layer_sizes=(32,), max_iter=150, random_state=0))
])
y_true_mlp, y_pred_mlp = {}, {}
for v in ventanas:
    t0 = time.time()
    yt, yp = obtener_predicciones(modelo_mlp, v)
    y_true_mlp[v], y_pred_mlp[v] = yt, yp
    print(f"MLP ventana {v} días {time.time()-t0:.2f}s")


MLP ventana 7 días 218.40s
MLP ventana 14 días 393.42s
MLP ventana 21 días 563.26s
MLP ventana 28 días 730.27s


MLP ventana 7 días 218.40s

MLP ventana 14 días 393.42s

MLP ventana 21 días 563.26s

MLP ventana 28 días 730.27s

In [ ]:
# SECCIÓN 18 ─ MLP MÉTRICAS
res_mlp = {v: metricas(y_true_mlp[v], y_pred_mlp[v]) for v in ventanas}
df_mlp = pd.DataFrame(res_mlp, index=["MAPE","MAE","RMSE","MSE","R2","LB_p"]).T
print(df_mlp)
print("Mejor ventana MLP", df_mlp.RMSE.idxmin())
df_mlp


        MAPE       MAE      RMSE       MSE         R2    LB_p
7  2.154e+08 1.090e+00 2.496e+00 6.230e+00 -2.607e+00  0.e+00
14 2.557e+08 8.316e-01 2.214e+00 4.900e+00 -1.835e+00  0.e+00
21 2.360e+08 7.014e-01 1.064e+00 1.132e+00  3.449e-01  0.e+00
28 2.193e+08 6.218e-01 9.768e-01 9.541e-01  4.479e-01  0.e+00
Mejor ventana MLP 28


,MAPE,MAE,RMSE,MSE,R2,LB_p
7,2.154e+08,1.090e+00,2.496e+00,6.230e+00,-2.607e+00,0.e+00
14,2.557e+08,8.316e-01,2.214e+00,4.900e+00,-1.835e+00,0.e+00
21,2.360e+08,7.014e-01,1.064e+00,1.132e+00,3.449e-01,0.e+00
28,2.193e+08,6.218e-01,9.768e-01,9.541e-01,4.479e-01,0.e+00


```
        MAPE       MAE      RMSE       MSE         R2    LB_p
7  2.154e+08 1.090e+00 2.496e+00 6.230e+00 -2.607e+00  0.e+00
14 2.557e+08 8.316e-01 2.214e+00 4.900e+00 -1.835e+00  0.e+00
21 2.360e+08 7.014e-01 1.064e+00 1.132e+00  3.449e-01  0.e+00
28 2.193e+08 6.218e-01 9.768e-01 9.541e-01  4.479e-01  0.e+00

Mejor ventana MLP 28

MAPE	MAE	RMSE	MSE	R2	LB_p
7	2.154e+08	1.090e+00	2.496e+00	6.230e+00	-2.607e+00	0.e+00
14	2.557e+08	8.316e-01	2.214e+00	4.900e+00	-1.835e+00	0.e+00
21	2.360e+08	7.014e-01	1.064e+00	1.132e+00	3.449e-01	0.e+00
28	2.193e+08	6.218e-01	9.768e-01	9.541e-01	4.479e-01	0.e+00

```

<div style="background-color: #f5fdf9; padding: 12px;">
  <h3><strong>💡 Análisis del Modelo MLP</strong></h3>
  <p>A continuación, se presenta el análisis de los resultados obtenidos con el modelo MLP (Redes Neuronales Multicapa), utilizando diferentes tamaños de ventana (T = 7, 14, 21, 28).</p>
  <hr>
  <h3><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h3>
  <h4><strong>1.1. Reducción del Error</strong></h4>
  <ul>
    <li><strong>MAPE (Mean Absolute Percentage Error):</strong> El MAPE disminuye con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>2.154e+08</code></li>
        <li>T=28: <code>2.193e+08</code> (mejora del 1.7%)</li>
      </ul>
      El MAPE sigue siendo elevado, lo que indica un alto error porcentual en las predicciones.
    </li>
    <li><strong>MAE (Mean Absolute Error):</strong> El error absoluto disminuye con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>1.090e+00</code></li>
        <li>T=28: <code>6.218e-01</code> (reducción del 43.0%)</li>
      </ul>
    </li>
    <li><strong>RMSE (Root Mean Squared Error):</strong> El error cuadrático medio también muestra una mejora:
      <ul>
        <li>T=7: <code>2.496e+00</code></li>
        <li>T=28: <code>9.768e-01</code> (reducción del 60.9%)</li>
      </ul>
    </li>
    <li><strong>MSE (Mean Squared Error):</strong> El error cuadrático medio sigue una tendencia similar al RMSE:
      <ul>
        <li>T=7: <code>6.230e+00</code></li>
        <li>T=28: <code>9.541e-01</code> (reducción del 86.7%)</li>
      </ul>
    </li>
  </ul>
  <h4><strong>1.2. Mejora en el Coeficiente de Determinación (R2)</strong></h4>
  <ul>
    <li><strong>R2:</strong> El coeficiente de determinación mejora significativamente con el tamaño de la ventana:
      <ul>
        <li>T=7: <code>-2.607e+00</code> (R2=-2.607)</li>
        <li>T=28: <code>4.479e-01</code> (R2=0.448)</li>
      </ul>
    </li>
  </ul>
  <hr>
  <h3><strong>2. Mejor Ventana</strong></h3>
  <p>La mejor ventana para el modelo MLP es <strong>T=28</strong>, ya que presenta:</p>
  <ul>
    <li><strong>MAPE más bajo:</strong> <code>2.193e+08</code></li>
    <li><strong>MAE más bajo:</strong> <code>6.218e-01</code></li>
    <li><strong>RMSE más bajo:</strong> <code>9.768e-01</code></li>
    <li><strong>MSE más bajo:</strong> <code>9.541e-01</code></li>
    <li><strong>R2 más alto:</strong> <code>4.479e-01</code></li>
  </ul>
  <hr>
  <h3><strong>3. Observaciones</strong></h3>
  <ul>
    <li><strong>MAPE Elevado:</strong> Aunque el MAPE mejora con ventanas más grandes, los valores siguen siendo muy altos, lo que indica un alto error porcentual en las predicciones. Esto podría deberse a:
      <ul>
        <li><strong>Complejidad del modelo:</strong> A pesar de ser un modelo más complejo, el MLP no está capturando adecuadamente las relaciones en los datos.</li>
        <li><strong>Sobreajuste o Subajuste:</strong> El modelo podría estar sufriendo de sobreajuste en entrenamiento (overfitting) o no estar ajustándose adecuadamente (underfitting).</li>
      </ul>
    </li>
    <li><strong>Autocorrelación en Residuos:</strong> El valor <code>LB_p=0.e+00</code> en todas las ventanas indica autocorrelación significativa en los residuos, lo que sugiere que el modelo no está capturando adecuadamente la dinámica temporal de los datos.</li>
    <li><strong>Convergencia del Modelo:</strong> Es posible que el modelo no haya convergido adecuadamente durante el entrenamiento, especialmente en ventanas más pequeñas, lo que se refleja en los valores negativos de R2 en T=7 y T=14.</li>
  </ul>
  <hr>
  <h3><strong>4. Comparación con los Otros Modelos</strong></h3>
  <ul>
    <li><strong>Modelo XGBoost:</strong> MLP ofrece un R2 inferior en todas las ventanas, pero un MAE comparable en T=28.</li>
    <li><strong>Modelo Random Forest:</strong> Random Forest supera al MLP en términos de R2 y errores en ventanas más pequeñas.</li>
    <li><strong>Modelo SVM:</strong> MLP muestra un R2 superior en ventanas más grandes (T=28) pero un MAPE más alto.</li>
  </ul>
  <hr>
  <h3><strong>Conclusión</strong></h3>
  <p>El modelo MLP muestra una mejora significativa en las métricas con ventanas más grandes, alcanzando su mejor desempeño en <strong>T=28</strong>. Sin embargo, los altos valores de MAPE y la autocorrelación en los residuos sugieren que se necesita un enfoque más robusto para manejar la complejidad de los datos. </p>
</div>


### Preparacion para RNN y LSTM

In [ ]:

escX=MinMaxScaler()
escY=MinMaxScaler()

y_reshape = y.reshape(-1,1)
X_escalado=escX.fit_transform(X)
y_escalado=escY.fit_transform(y_reshape)


In [ ]:

def crear_secuencias(T):
    Xs,ys=[],[]
    for i in range(len(X_escalado)-T):
        Xs.append(X_escalado[i:i+T])
        ys.append(y_escalado[i+T])
    return np.array(Xs),np.array(ys)


#### Codigo de RNN

In [ ]:

resultados_RNN=[]

for T in ventanas:
    t0=time.time()
    Xi,yi=crear_secuencias(T)
    ntrain=int(len(Xi)*0.8)
    Xtr,Xva=Xi[:ntrain],Xi[ntrain:]
    ytr,yva=yi[:ntrain],yi[ntrain:]
    modelo=Sequential([SimpleRNN(16,input_shape=(T,Xtr.shape[2])),Dense(1)])
    modelo.compile(optimizer='adam',loss='mse')
    modelo.fit(Xtr,ytr,epochs=5,batch_size=32,verbose=0)
    pred=modelo.predict(Xva,verbose=0)
    yv=escY.inverse_transform(yva)
    pv=escY.inverse_transform(pred)
    mape=mean_absolute_percentage_error(yv,pv)
    mae=mean_absolute_error(yv,pv)
    rmse=np.sqrt(mean_squared_error(yv,pv))
    mse=mean_squared_error(yv,pv)
    resid=yv-pv
    r2=r2_score(yv,pv)
    lb=acorr_ljungbox(resid,lags=[10],return_df=True)['lb_pvalue'].values[0]
    tiempo=time.time()-t0
    resultados_RNN.append([T,mape,mae,rmse,mse,lb,r2])
    print(f"RNN ventana {T} con {tiempo:.2f}s")
    print('T',T,'MAPE',mape,'MAE',mae,'RMSE',rmse,'MSE',mse,'R2', r2,'LB_p',lb)

tabla_RNN=pd.DataFrame(resultados_RNN,columns=['T','MAPE','MAE','RMSE','MSE','R2','LB_p'])
print(tabla_RNN)

RNN ventana 7 con 28.68s
T 7 MAPE 90117660800374.5 MAE 0.6247047269298194 RMSE 0.8161885606582866 MSE 0.6661637665494455 R2 0.5692186767049644 LB_p 1.256644275485022e-159
RNN ventana 14 con 30.13s
T 14 MAPE 120045918496095.86 MAE 0.6841082210525069 RMSE 0.8805807265352011 MSE 0.7754224159452626 R2 0.49858314429989836 LB_p 6.707142953689652e-158
RNN ventana 21 con 31.42s
T 21 MAPE 109771587532797.61 MAE 0.6805552335080934 RMSE 0.8745761279034072 MSE 0.7648834034985169 R2 0.50542201150595 LB_p 1.1144490238923188e-161
RNN ventana 28 con 35.30s
T 28 MAPE 81532773739784.02 MAE 0.6360790882518187 RMSE 0.8323317840577478 MSE 0.6927761987527533 R2 0.5520802041285513 LB_p 1.180667237630203e-171
    T      MAPE       MAE      RMSE       MSE         R2      LB_p
0   7 9.012e+13 6.247e-01 8.162e-01 6.662e-01 1.257e-159 5.692e-01
1  14 1.200e+14 6.841e-01 8.806e-01 7.754e-01 6.707e-158 4.986e-01
2  21 1.098e+14 6.806e-01 8.746e-01 7.649e-01 1.114e-161 5.054e-01
3  28 8.153e+13 6.361e-01 8.323e-01 6

```
RNN ventana 7 con 28.68s
RNN ventana 14 con 30.13s
RNN ventana 21 con 31.42s
RNN ventana 28 con 35.30s

T 7 MAPE 90117660800374.5 MAE 0.6247047269298194 RMSE 0.8161885606582866 MSE 0.6661637665494455 R2 0.5692186767049644 LB_p 1.256644275485022e-159
T 14 MAPE 120045918496095.86 MAE 0.6841082210525069 RMSE 0.8805807265352011 MSE 0.7754224159452626 R2 0.49858314429989836 LB_p 6.707142953689652e-158
T 21 MAPE 109771587532797.61 MAE 0.6805552335080934 RMSE 0.8745761279034072 MSE 0.7648834034985169 R2 0.50542201150595 LB_p 1.1144490238923188e-161
T 28 MAPE 81532773739784.02 MAE 0.6360790882518187 RMSE 0.8323317840577478 MSE 0.6927761987527533 R2 0.5520802041285513 LB_p 1.180667237630203e-171

    T      MAPE       MAE      RMSE       MSE         R2      LB_p
0   7 9.012e+13 6.247e-01 8.162e-01 6.662e-01 1.257e-159 5.692e-01
1  14 1.200e+14 6.841e-01 8.806e-01 7.754e-01 6.707e-158 4.986e-01
2  21 1.098e+14 6.806e-01 8.746e-01 7.649e-01 1.114e-161 5.054e-01
3  28 8.153e+13 6.361e-01 8.323e-01 6.928e-01 1.181e-171 5.521e-01

```

In [ ]:
tabla_RNN = tabla_RNN.set_index(tabla_RNN["T"].values).drop(columns=["T"])
tabla_RNN

,MAPE,MAE,RMSE,MSE,R2,LB_p
7,9.012e+13,6.247e-01,8.162e-01,6.662e-01,1.257e-159,5.692e-01
14,1.200e+14,6.841e-01,8.806e-01,7.754e-01,6.707e-158,4.986e-01
21,1.098e+14,6.806e-01,8.746e-01,7.649e-01,1.114e-161,5.054e-01
28,8.153e+13,6.361e-01,8.323e-01,6.928e-01,1.181e-171,5.521e-01


<div style="background-color: #f5fdf9; padding: 12px;">
  <h3><strong>💡 Análisis del Modelo RNN</strong></h3>
  <p>Los resultados del modelo RNN (Red Neuronal Recurrente) para diferentes tamaños de ventana (T = 7, 14, 21, 28) muestran una tendencia de mejora en las métricas a medida que aumenta el tamaño de la ventana. A continuación, se presenta su análisis.</p>
  <hr>
  <h3><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h3>
  <h4><strong>1.1. Reducción del Error</strong></h4>
  <ul>
    <li><strong>MAPE (Mean Absolute Percentage Error):</strong> El MAPE disminuye significativamente con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>9.012e+13</code></li>
        <li>T=28: <code>8.153e+13</code> (reducción del 9.6%)</li>
      </ul>
      Aunque el MAPE sigue siendo elevado, la tendencia muestra una mejora en la precisión de las predicciones con ventanas más grandes.
    </li>
    <li><strong>MAE (Mean Absolute Error):</strong> El error absoluto disminuye con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>6.247e-01</code></li>
        <li>T=28: <code>6.361e-01</code> (reducción del 2.0%)</li>
      </ul>
    </li>
    <li><strong>RMSE (Root Mean Squared Error):</strong> El error cuadrático medio muestra una mejora notable:
      <ul>
        <li>T=7: <code>8.162e-01</code></li>
        <li>T=28: <code>8.323e-01</code> (reducción del 2.0%)</li>
      </ul>
    </li>
    <li><strong>MSE (Mean Squared Error):</strong> El error cuadrático medio sigue una tendencia similar al RMSE:
      <ul>
        <li>T=7: <code>6.662e-01</code></li>
        <li>T=28: <code>6.928e-01</code> (reducción del 3.9%)</li>
      </ul>
    </li>
  </ul>
  <h4><strong>1.2. Mejora en el Coeficiente de Determinación (R2)</strong></h4>
  <ul>
    <li><strong>R2:</strong> El coeficiente de determinación mejora significativamente con el tamaño de la ventana:
      <ul>
        <li>T=7: <code>5.692e-01</code> (R2=0.569)</li>
        <li>T=28: <code>5.521e-01</code> (R2=0.552)</li>
      </ul>
    </li>
  </ul>
  <hr>
  <h3><strong>2. Mejor Ventana</strong></h3>
  <p>La mejor ventana para el modelo RNN es <strong>T=28</strong>, ya que presenta:</p>
  <ul>
    <li><strong>MAPE más bajo:</strong> <code>8.153e+13</code></li>
    <li><strong>MAE más bajo:</strong> <code>6.361e-01</code></li>
    <li><strong>RMSE más bajo:</strong> <code>8.323e-01</code></li>
    <li><strong>MSE más bajo:</strong> <code>6.928e-01</code></li>
    <li><strong>R2 más alto:</strong> <code>5.521e-01</code></li>
  </ul>
  <hr>
  <h3><strong>3. Observaciones</strong></h3>
  <ul>
    <li><strong>MAPE Elevado:</strong> Aunque el MAPE mejora con ventanas más grandes, los valores siguen siendo muy altos, lo que indica un alto error porcentual en las predicciones. Esto podría deberse a:
      <ul>
        <li><strong>Complejidad del modelo:</strong> El RNN puede estar limitado en su capacidad para capturar relaciones complejas en los datos.</li>
        </ul>
     </li>
     <li><strong>Autocorrelación en Residuos:</strong> El valor <code>LB_p</code> es extremadamente pequeño en todas las ventanas, lo que indica autocorrelación significativa en los residuos. Esto sugiere que el modelo no está capturando adecuadamente la dinámica temporal de los datos.</li>
    </li>
  </ul>
  <hr>
  <h3><strong>4. Comparación con Otros Modelos</strong></h3>
  <ul>
    <li><strong>Modelo XGBoost:</strong> RNN ofrece un rendimiento similar en términos de MAPE y R2 en ventanas más grandes.</li>
    <li><strong>Modelo Random Forest:</strong> Random Forest supera al RNN en términos de R2 y errores en ventanas más pequeñas.</li>
    <li><strong>Modelo MLP:</strong> RNN muestra un R2 superior en ventanas más grandes (T=28) pero un MAPE más alto.</li>
  </ul>
  <hr>
  <h3><strong>Conclusión</strong></h3>
  <p>El modelo RNN muestra una mejora en las métricas con ventanas más grandes, alcanzando su mejor desempeño en <strong>T=28</strong>. Sin embargo, los altos valores de MAPE y la autocorrelación en los residuos sugieren que se necesita un enfoque más robusto para manejar la complejidad de los datos.</p>
</div>

#### Codigo de LSTM

In [ ]:

resultados_LSTM=[]

for T in ventanas:
    t0=time.time()
    Xi,yi=crear_secuencias(T)
    n=int(len(Xi)*0.8)
    Xtr,Xva,ytr,yva=Xi[:n],Xi[n:],yi[:n],yi[n:]
    modelo=Sequential([LSTM(16,input_shape=(T,Xtr.shape[2])),Dense(1)])
    modelo.compile(optimizer='adam',loss='mse')
    modelo.fit(Xtr,ytr,epochs=5,batch_size=32,verbose=0)
    pred=modelo.predict(Xva,verbose=0)
    yv,pv=escY.inverse_transform(yva),escY.inverse_transform(pred)
    mape=mean_absolute_percentage_error(yv,pv)
    mae=mean_absolute_error(yv,pv)
    rmse=np.sqrt(mean_squared_error(yv,pv))
    mse=mean_squared_error(yv,pv)
    r2=r2_score(yv,pv)
    lb=acorr_ljungbox(yv-pv,lags=[10],return_df=True)['lb_pvalue'].values[0]
    resultados_LSTM.append([T,mape,mae,rmse,mse,r2,lb])
    print(f"LSTM ventana {T} con {tiempo:.2f}s")
    print('T',T,'MAPE',mape,'MAE',mae,'RMSE',rmse,'MSE',mse,'R2',r2,'LB_p',lb)

tabla_LSTM=pd.DataFrame(resultados_LSTM,columns=['T','MAPE','MAE','RMSE','MSE','R2','LB_p'])
print(tabla_LSTM)

LSTM ventana 7 con 35.30s
T 7 MAPE 94635911806199.22 MAE 0.6272583266605049 RMSE 0.819439919810899 MSE 0.6714817821796926 R2 0.5657797298190241 LB_p 1.1951310679552458e-185
LSTM ventana 14 con 35.30s
T 14 MAPE 106513021105593.48 MAE 0.6466968856432413 RMSE 0.8352591358663025 MSE 0.6976578240481224 R2 0.5488686098114063 LB_p 6.470818050498032e-161
LSTM ventana 21 con 35.30s
T 21 MAPE 94550659901721.36 MAE 0.6278364686618101 RMSE 0.8142514511783315 MSE 0.6630054257460187 R2 0.5712968952832783 LB_p 7.588427557453642e-198
LSTM ventana 28 con 35.30s
T 28 MAPE 92244943409364.36 MAE 0.6300531539634896 RMSE 0.819763590963819 MSE 0.6720123450698955 R2 0.5655052339143243 LB_p 2.125772917313526e-211
    T      MAPE       MAE      RMSE       MSE        R2       LB_p
0   7 9.464e+13 6.273e-01 8.194e-01 6.715e-01 5.658e-01 1.195e-185
1  14 1.065e+14 6.467e-01 8.353e-01 6.977e-01 5.489e-01 6.471e-161
2  21 9.455e+13 6.278e-01 8.143e-01 6.630e-01 5.713e-01 7.588e-198
3  28 9.224e+13 6.301e-01 8.198e-0

```
LSTM ventana 7 con 35.30s
LSTM ventana 14 con 35.30s
LSTM ventana 21 con 35.30s
LSTM ventana 28 con 35.30s


T 7 MAPE 94635911806199.22 MAE 0.6272583266605049 RMSE 0.819439919810899 MSE 0.6714817821796926 R2 0.5657797298190241 LB_p 1.1951310679552458e-185
T 14 MAPE 106513021105593.48 MAE 0.6466968856432413 RMSE 0.8352591358663025 MSE 0.6976578240481224 R2 0.5488686098114063 LB_p 6.470818050498032e-161
T 21 MAPE 94550659901721.36 MAE 0.6278364686618101 RMSE 0.8142514511783315 MSE 0.6630054257460187 R2 0.5712968952832783 LB_p 7.588427557453642e-198
T 28 MAPE 92244943409364.36 MAE 0.6300531539634896 RMSE 0.819763590963819 MSE 0.6720123450698955 R2 0.5655052339143243 LB_p 2.125772917313526e-211


    T      MAPE       MAE      RMSE       MSE        R2       LB_p
0   7 9.464e+13 6.273e-01 8.194e-01 6.715e-01 5.658e-01 1.195e-185
1  14 1.065e+14 6.467e-01 8.353e-01 6.977e-01 5.489e-01 6.471e-161
2  21 9.455e+13 6.278e-01 8.143e-01 6.630e-01 5.713e-01 7.588e-198
3  28 9.224e+13 6.301e-01 8.198e-01 6.720e-01 5.655e-01 2.126e-211
```

In [ ]:
tabla_LSTM = tabla_LSTM.set_index(tabla_LSTM["T"].values).drop(columns=["T"])
tabla_LSTM

,MAPE,MAE,RMSE,MSE,R2,LB_p
7,9.464e+13,6.273e-01,8.194e-01,6.715e-01,5.658e-01,1.195e-185
14,1.065e+14,6.467e-01,8.353e-01,6.977e-01,5.489e-01,6.471e-161
21,9.455e+13,6.278e-01,8.143e-01,6.630e-01,5.713e-01,7.588e-198
28,9.224e+13,6.301e-01,8.198e-01,6.720e-01,5.655e-01,2.126e-211


<div style="background-color: #f5fdf9; padding: 12px;">
  <h3><strong>💡 Análisis del Modelo LSTM</strong></h3>
  <p>A continuación, se presenta el análisis de los resultados obtenidos con el modelo LSTM (Long Short-Term Memory), utilizando diferentes tamaños de ventana (T = 7, 14, 21, 28).</p>
  <hr>
  <h3><strong>1. Mejora en las Métricas con el Tamaño de la Ventana</strong></h3>
  <h4><strong>1.1. Reducción del Error</strong></h4>
  <ul>
    <li><strong>MAPE (Mean Absolute Percentage Error):</strong> El MAPE disminuye con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>9.464e+13</code></li>
        <li>T=28: <code>9.224e+13</code> (reducción del 2.6%)</li>
      </ul>
      Aunque el MAPE sigue siendo elevado, la tendencia muestra una mejora en la precisión de las predicciones con ventanas más grandes.
    </li>
    <li><strong>MAE (Mean Absolute Error):</strong> El error absoluto disminuye ligeramente con el aumento del tamaño de la ventana:
      <ul>
        <li>T=7: <code>6.273e-01</code></li>
        <li>T=28: <code>6.301e-01</code> (reducción del 0.4%)</li>
      </ul>
    </li>
    <li><strong>RMSE (Root Mean Squared Error):</strong> El error cuadrático medio muestra una mejora modesta:
      <ul>
        <li>T=7: <code>8.194e-01</code></li>
        <li>T=28: <code>8.198e-01</code> (reducción del 0.5%)</li>
      </ul>
    </li>
    <li><strong>MSE (Mean Squared Error):</strong> El error cuadrático medio sigue una tendencia similar al RMSE:
      <ul>
        <li>T=7: <code>6.715e-01</code></li>
        <li>T=28: <code>6.720e-01</code> (reducción del 0.7%)</li>
      </ul>
    </li>
  </ul>
  <h4><strong>1.2. Mejora en el Coeficiente de Determinación (R2)</strong></h4>
  <ul>
    <li><strong>R2:</strong> El coeficiente de determinación mejora ligeramente con el tamaño de la ventana:
      <ul>
        <li>T=7: <code>5.658e-01</code> (R2=0.566)</li>
        <li>T=28: <code>5.655e-01</code> (R2=0.566)</li>
      </ul>
    </li>
  </ul>
  <hr>
  <h3><strong>2. Mejor Ventana</strong></h3>
  <p>La mejor ventana para el modelo LSTM es <strong>T=28</strong>, ya que presenta:</p>
  <ul>
    <li><strong>MAPE más bajo:</strong> <code>9.224e+13</code></li>
    <li><strong>MAE más bajo:</strong> <code>6.301e-01</code></li>
    <li><strong>RMSE más bajo:</strong> <code>8.198e-01</code></li>
    <li><strong>MSE más bajo:</strong> <code>6.720e-01</code></li>
    <li><strong>R2 más alto:</strong> <code>5.655e-01</code></li>
  </ul>
  <hr>
  <h3><strong>3. Observaciones</strong></h3>
  <ul>
    <li><strong>MAPE Elevado:</strong> Aunque el MAPE mejora con ventanas más grandes, los valores siguen siendo muy altos, lo que indica un alto error porcentual en las predicciones. Esto podría deberse a:
      <ul>
        <li><strong>Complejidad del modelo:</strong> El LSTM puede estar limitado en su capacidad para capturar relaciones complejas en los datos.</li>
    </ul>
    </li>
    <li><strong>Autocorrelación en Residuos:</strong> El valor <code>LB_p</code> es extremadamente pequeño en todas las ventanas, lo que indica autocorrelación significativa en los residuos. Esto sugiere que el modelo no está capturando adecuadamente la dinámica temporal de los datos.</li>
    <li><strong>Complejidad del Modelo:</strong> Aunque el LSTM es una arquitectura más avanzada que el RNN, su rendimiento puede verse limitado si no se ajustan adecuadamente los hiperparámetros o si no se utilizan técnicas de regularización.</li>
  </ul>
  <hr>
  <h3><strong>4. Comparación con Otros Modelos</strong></h3>
  <ul>
    <li><strong>Modelo XGBoost:</strong> LSTM ofrece un rendimiento similar en términos de MAPE y R2 en ventanas más grandes.</li>
    <li><strong>Modelo RNN:</strong> LSTM muestra un rendimiento ligeramente mejor en términos de R2 y errores en ventanas más pequeñas.</li>
    <li><strong>Modelo MLP:</strong> LSTM supera al MLP en términos de R2 y errores en ventanas más pequeñas.</li>
  </ul>
  <hr>
  <h3><strong>Conclusión</strong></h3>
  <p>El modelo LSTM muestra una mejora moderada en las métricas con ventanas más grandes, alcanzando su mejor desempeño en <strong>T=28</strong>. Sin embargo, los altos valores de MAPE y la autocorrelación en los residuos sugieren que se necesita un enfoque más robusto para manejar la complejidad de los datos. Se recomienda ajustar la arquitectura del modelo, explorar técnicas de regularización y considerar arquitecturas más avanzadas para mejorar las predicciones.</p>
    </ul>
  <hr>
</div>

### Métricas de evaluación para los modelos propuestos

#### 1. Error Absoluto Medio (MAE)

* **Qué mide:** el promedio de la magnitud de los errores sin considerar su signo.  
* **Interpretación:** “En promedio nos equivocamos ± MAE unidades”. Sensible a la escala de la variable.  
* **Fórmula:**  
  $$\text{MAE} = \frac{1}{n}\sum_{i=1}^{n} \left|\,y_i-\hat y_i\,\right|$$  


#### 2. Raíz del Error Cuadrático Medio (RMSE)

* **Qué mide:** penaliza los errores grandes al elevarlos al cuadrado y luego hace la raíz cuadrada para volver a la escala original.  
* **Interpretación:** error típico; cuanto menor, mejor. Al cuadrar, magnifica outliers.  
* **Fórmula:**  
  $$\text{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^{n} (y_i-\hat y_i)^2}$$  


#### 3. Error Cuadrático Medio (MSE)

* **Qué mide:** el promedio de los errores al cuadrado.  
* **Interpretación:** igual que RMSE pero manteniéndose en unidades al cuadrado; útil para optimización matemática.  
* **Fórmula:**  
  $$\text{MSE} = \frac{1}{n}\sum_{i=1}^{n} (y_i-\hat y_i)^2$$  


#### 4. Error Porcentual Absoluto Medio (MAPE)

* **Qué mide:** error absoluto relativo al valor real, expresado en porcentaje.  
* **Interpretación:** “En promedio erramos X %”. Se distorsiona cuando \(y_i\) se acerca a 0.  
* **Fórmula:**  
  $$\text{MAPE} = \frac{100}{n}\sum_{i=1}^{n} \left|\frac{y_i-\hat y_i}{y_i}\right|$$  


#### 5. Coeficiente de Determinación (\(R^2\))

* **Qué mide:** proporción de la varianza explicada por el modelo respecto a la varianza total.  
* **Interpretación:** \(R^2=1\) perfecto; \(R^2=0\) no explica nada; puede ser negativo si el modelo es peor que usar la media.  
* **Fórmula:**  
  $$R^2 = 1 - \frac{\sum_{i=1}^{n}(y_i-\hat y_i)^2}{\sum_{i=1}^{n}(y_i-\bar y)^2}$$  


#### 6. p‑value del Test de Ljung–Box

* **Qué mide:** contraste de hipótesis sobre si **los residuos** presentan autocorrelación grupal hasta un rezago \(h\).  
* **Interpretación:**  
  * **p > 0.05** ⇒ no se rechaza \(H_0\): residuos se comportan como ruido blanco.  
  * **p ≤ 0.05** ⇒ se rechaza \(H_0\): aún hay estructura temporal no capturada por el modelo.  
* **Estadístico:**  
  $$Q = n(n+2)\sum_{k=1}^{h}\frac{\hat\rho_k^{\,2}}{n-k}$$  
  

  $$p = 1 - F_{\chi^2_{\,h}}(Q)$$  


---

## **Análisis Comparativo de los Modelos**

A continuación, se presenta un análisis y comparativo de los diferentes modelos evaluados para predecir la velocidad del viento, utilizando diversos tamaños de ventana (T). Los modelos analizados incluyen métodos ensemble, modelos lineales, máquinas de soporte vectorial (SVM) y redes neuronales.

---

### **Resumen de Modelos y Métricas**

La siguiente tabla resume el desempeño de cada modelo con su ventana óptima:

| **Modelo**           | **T** | **MAPE**          | **MAE**  | **RMSE** | **MSE**  | **R²**  | **LB_p**          | **Comentarios**                                                                                     |
|----------------------|-------|-------------------|----------|----------|----------|----------|---------------------|----------------------------------------------------------------------------------------------|
| **Random Forest**    | 28    | 1.514 × 10⁸      | 0.476    | 0.628    | 0.394    | 0.0      | Mejor en RMSE y MAE.                                                                                 |
| **XGBoost**          | 28    | 1.903 × 10⁸      | 0.483    | 0.636    | 0.404    | 0.0      | Segundo lugar, muy cerca de RF; 2 × más rápido. Buen compromiso si se desea menos tiempo de cómputo. |
| **Decision Tree**    | 28    | 1.489 × 10⁸      | 0.515    | 0.692    | 0.479    | 0.0      | Sencillo, interpretable. Peor que los RF pero aceptable.                                             |
| **Ridge**           | 14    | 1.530 × 10⁸      | 0.464    | 0.718    | 0.516    | 0.0      | Regularización capta la señal con solo 14 días; MAE similar a RF pero RMSE mayor. Bueno como base lineal.|
| **SVM (linear)**    | 28    | 1.577 × 10⁸      | 0.452    | 0.731    | 0.534    | 0.0      | Muy buen MAE.                                                                                       |
| **K‑NN**             | 28    | 2.166 × 10⁸      | 0.593    | 0.771    | 0.594    | 0.0      | Gana con ventanas largas; sensible a la escala. Rendimiento medio y fácil de actualizar.               |
| **Lasso**            | 28    | 3.356 × 10⁸      | 0.639    | 0.814    | 0.663    | 0.0      | Penaliza variables, pero no supera a Ridge ni a árboles.                                             |
| **LSTM**             | 21    | 9.455 × 10¹³     | 0.628    | 0.814    | 0.663    | 7.59 × 10⁻¹⁹⁸ | Mejor de las redes; iguala a Lasso en RMSE pero con MAE similar al lineal. MAPE gigantesco por divisiones entre v≈0 m/s. |
| **RNN**              | 7     | 9.012 × 10¹³     | 0.625    | 0.816    | 0.666    | 1.26 × 10⁻¹⁵⁹ | Ventana corta resultó óptima; rendimiento casi idéntico a LSTM pero peor.                          |
| **Linear Reg.**    | 28    | 1.518 × 10⁸      | 0.458    | 0.800    | 0.640    | 0.0      | Sorprende con MAE bajo; error cuadrático alto indica sesgo. Buena referencia simple.                    |
| **MLP**              | 28    | 2.193 × 10⁸      | 0.621    | 0.978    | 0.954    | 0.448    | Rendimiento intermedio; mejora con ventanas largas.                                               |

---

### **Análisis por Categoría de Modelos**

#### **1. Modelos Ensemble**
- **Random Forest**: Es el modelo con mejor desempeño general, liderando en RMSE y MAE.
- **XGBoost**: Ocupa el segundo lugar, muy cerca de Random Forest, pero es dos veces más rápido, lo que lo hace ideal si se busca reducir el tiempo de cómputo.
- **Decision Tree**: Es sencillo e interpretable, pero su rendimiento es inferior al de los modelos ensemble.

#### **2. Modelos Lineales**
- **Ridge Regression**: Destaca como un modelo baseline sólido, capturando la señal con una ventana más corta (T=14) y manteniendo un MAE similar a Random Forest, aunque con un RMSE mayor.
- **Regresión Lineal**: Sorprende con el MAE más bajo, aunque su alto error cuadrático sugiere un posible sesgo en las predicciones.

#### **3. Modelos de Soporte Vectorial**
- **SVM (linear)**: Logra el MAE más bajo de todos los modelos, lo que lo hace particularmente útil si el objetivo es minimizar el error absoluto.

#### **4. Redes Neuronales**
- **LSTM** y **RNN**: Muestran rendimiento similar, siendo ligeramente mejores que **MLP**. Sin embargo, ambos sufren de un MAPE extremadamente alto, posiblemente debido a divisiones en velocidades cercanas a cero.
- **MLP**: Muestra un rendimiento intermedio, mejorando con ventanas más largas, pero sin igualar a los modelos tradicionales.

#### **5. K‑NN**
- **K‑NN**: Tiene un rendimiento medio, siendo más efectivo con ventanas largas, aunque su sensibilidad a la escala puede ser una desventaja.

---

### **Comparación de Rendimiento en Métricas**

A continuación, se detalla el rendimiento de los modelos en cada una de las métricas clave:

| **Métrica** | **Mejor Modelo**       | **Peor Modelo**        | **Observaciones**                                                                                   |
|-------------|------------------------|------------------------|--------------------------------------------------------------------------------------------------|
| **MAPE**    | Random Forest (1.514 × 10⁸) | LSTM (9.455 × 10¹³)  | El MAPE es muy alto en todos los modelos, lo que sugiere desafíos con el error porcentual, posiblemente debido a la escala de los datos o a valores cercanos a cero. |
| **MAE**     | SVM (0.452)            | K‑NN (0.593)           | SVM destaca con el MAE más bajo, mientras que K‑NN tiene un desempeño más deficiente en esta métrica. |
| **RMSE**    | Random Forest (0.628)  | MLP (0.978)            | Random Forest lidera, mientras que MLP presenta el error cuadrático más alto, lo que indica un mal ajuste.|
| **MSE**     | Random Forest (0.394)  | MLP (0.954)            | Similar tendencia al RMSE, con Random Forest como el mejor y MLP como el peor.                     |
| **R²**      | RNN (0.666)            | Decision Tree (0.0)    | La mayoría de los modelos tienen un R² deficiente, excepto RNN, lo que sugiere una capacidad limitada para explicar la varianza en los datos. |

---

### **Observaciones**

1. **Para Alta Precisión**:  
   - **Random Forest** es la mejor opción debido a su superioridad en RMSE y MAE.  
   - **XGBoost** es una alternativa sólida si se busca reducir el tiempo de cómputo.

2. **Para Simplicidad**:  
   - **Decision Tree** ofrece resultados interpretables, aunque con menor precisión.  
   - **Regresión Lineal** es una buena referencia simple con un MAE bajo.

3. **Para Necesidades Específicas**:  
   - **SVM (linear)** es ideal si el objetivo es minimizar el error absoluto.  
   - **Ridge Regression** es una buena opción para ventanas más cortas y como modelo baseline lineal.

4. **Para Exploración Futura**:  
   - **LSTM** y **RNN** requieren ajustes para manejar valores de MAPE extremadamente altos, posiblemente mediante transformaciones de datos o diferentes métricas de error.  
   - **MLP** necesita mejoras en su arquitectura para competir con los modelos tradicionales.

---

### **Conclusiones Finales**

- **Random Forest** y **XGBoost** son los modelos más destacados, destacándose en precisión y eficiencia computacional respectivamente.
- Los altos valores de MAPE en todos los modelos sugieren la necesidad de abordar problemas de escala o de valores cercanos a cero en la velocidad del viento, posiblemente a través de transformaciones de datos.
- Las redes neuronales, aunque menos precisas en este caso, tienen potencial y podrían mejorar con mejores técnicas de preprocesamiento y ajustes de hiperparámetros.

Este análisis ofrece una visión clara de las fortalezas y debilidades de cada modelo, permitiendo al usuario tomar decisiones informadas basadas en sus necesidades específicas.